## Business Understanding
Objective: Predict the unit sales for thousands of items sold at different Favorita stores.

Business Goals:

1. Ensure optimal stock levels at all stores.

2. Improve demand forecasting accuracy.

3. Understand factors affecting sales (promotions, holidays, oil prices, etc.).

Analytical Questions:

1. What are the dates with the lowest and highest sales each year?

2. How do sales compare across different months and years?

3. Did the earthquake in 2016 impact sales?

4. Are sales patterns different across store clusters, cities, states, and types?

5. What is the impact of wage payments on store sales?

## Data Understanding

In [73]:
import pyodbc     
from dotenv import dotenv_values    #import the dotenv_values function from the dotenv package
from dotenv import load_dotenv
import pandas as pd
import warnings 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from plotly.subplots import make_subplots
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import plotly.graph_objects as go


warnings.filterwarnings('ignore')

In [74]:
# Load environment variables from .env file
load_dotenv()
environment_variables=dotenv_values('.env')# Get the values for the credentials you set in the '.env' file
database=environment_variables.get("DATABASE")
server=environment_variables.get("SERVER")
username=environment_variables.get("USERNAME")
password=environment_variables.get("PASSWORD")

# Specify port if necessary (default SQL Server port is 1433)
port = 1433
server_with_port = f"{server},{port}"

# Update connection string to use ODBC Driver 17 for SQL Server
connection_string = (
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER={dap-projects-database.database.windows.net};"
    "DATABASE={dapDB};"
    "UID={learning_project_3};"
    "PWD={A$uB1Lp3$2@24}"
)

 
try:
    # Establish connection to the database
    conn = pyodbc.connect(connection_string)
    cursor = conn.cursor()

    # Example query to test the connection
    cursor.execute("SELECT TOP 10 * FROM dbo.oil")
    rows = cursor.fetchall()

    for row in rows:
        print(row)

except pyodbc.Error as ex:
    print("OperationalError:")
    print("Arguments:", ex.args)
    print("Connection String:", connection_string)

(datetime.date(2013, 1, 1), None)
(datetime.date(2013, 1, 2), 93.13999938964844)
(datetime.date(2013, 1, 3), 92.97000122070312)
(datetime.date(2013, 1, 4), 93.12000274658203)
(datetime.date(2013, 1, 7), 93.19999694824219)
(datetime.date(2013, 1, 8), 93.20999908447266)
(datetime.date(2013, 1, 9), 93.08000183105469)
(datetime.date(2013, 1, 10), 93.80999755859375)
(datetime.date(2013, 1, 11), 93.5999984741211)
(datetime.date(2013, 1, 14), 94.2699966430664)


In [75]:
query = "SELECT * FROM dbo.oil"

oil = pd.read_sql(query, conn)
oil.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.139999
2,2013-01-03,92.970001
3,2013-01-04,93.120003
4,2013-01-07,93.199997


In [76]:
query = "SELECT * FROM dbo.holidays_events"

holidays_events = pd.read_sql(query, conn)
holidays_events.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


In [77]:
query = "SELECT * FROM dbo.stores"

stores = pd.read_sql(query, conn)
stores.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [78]:
# Loading other datasets
train = pd.read_csv(r'C:\Users\magyir\Downloads\store-sales-forecasting\train.csv', parse_dates=['date'])
transactions = pd.read_csv(r"C:\Users\magyir\Downloads\store-sales-forecasting\transactions.csv", parse_dates=['date'])
sample_submission = pd.read_csv(r"C:\Users\magyir\Downloads\sample_submission.csv")
test = pd.read_csv(r"C:\Users\magyir\Downloads\test.csv", parse_dates=['date'])

In [79]:
# Ensuring date columns are in datetime format
train['date'] = pd.to_datetime(train['date'])
test['date'] = pd.to_datetime(test['date'])
transactions['date'] = pd.to_datetime(transactions['date'])
oil['date'] = pd.to_datetime(oil['date'])
holidays_events['date'] = pd.to_datetime(holidays_events['date'])

# Merge datasets
train = train.merge(stores, on='store_nbr', how='left')
train = train.merge(oil, on='date', how='left')
train = train.merge(transactions, on=['date', 'store_nbr'], how='left')
train = train.merge(holidays_events, on='date', how='left')



In [80]:
#view first 5 rows for merged data
train.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,type_x,cluster,dcoilwtico,transactions,type_y,locale,locale_name,description,transferred
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,NaN,NaN,Holiday,National,Ecuador,Primer dia del ano,False
1,1,2013-01-01,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,NaN,NaN,Holiday,National,Ecuador,Primer dia del ano,False
2,2,2013-01-01,1,BEAUTY,0.0,0,Quito,Pichincha,D,13,NaN,NaN,Holiday,National,Ecuador,Primer dia del ano,False
3,3,2013-01-01,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13,NaN,NaN,Holiday,National,Ecuador,Primer dia del ano,False
4,4,2013-01-01,1,BOOKS,0.0,0,Quito,Pichincha,D,13,NaN,NaN,Holiday,National,Ecuador,Primer dia del ano,False


In [81]:
#view first 5 rows for merged data
train.tail()

,id,date,store_nbr,family,sales,onpromotion,city,state,type_x,cluster,dcoilwtico,transactions,type_y,locale,locale_name,description,transferred
3054343,3000883,2017-08-15,9,POULTRY,438.133,0,Quito,Pichincha,B,6,47.57,2155.0,Holiday,Local,Riobamba,Fundacion de Riobamba,False
3054344,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,Quito,Pichincha,B,6,47.57,2155.0,Holiday,Local,Riobamba,Fundacion de Riobamba,False
3054345,3000885,2017-08-15,9,PRODUCE,2419.729,148,Quito,Pichincha,B,6,47.57,2155.0,Holiday,Local,Riobamba,Fundacion de Riobamba,False
3054346,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,Quito,Pichincha,B,6,47.57,2155.0,Holiday,Local,Riobamba,Fundacion de Riobamba,False
3054347,3000887,2017-08-15,9,SEAFOOD,16.000,0,Quito,Pichincha,B,6,47.57,2155.0,Holiday,Local,Riobamba,Fundacion de Riobamba,False


In [82]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3054348 entries, 0 to 3054347
Data columns (total 17 columns):
 #   Column        Dtype         
---  ------        -----         
 0   id            int64         
 1   date          datetime64[ns]
 2   store_nbr     int64         
 3   family        object        
 4   sales         float64       
 5   onpromotion   int64         
 6   city          object        
 7   state         object        
 8   type_x        object        
 9   cluster       int64         
 10  dcoilwtico    float64       
 11  transactions  float64       
 12  type_y        object        
 13  locale        object        
 14  locale_name   object        
 15  description   object        
 16  transferred   object        
dtypes: datetime64[ns](1), float64(3), int64(4), object(9)
memory usage: 396.1+ MB


Data Cleaning

In [83]:
# Identify missing values
missing_values = train.isnull().sum()
print("Missing values before cleaning:")
print(missing_values[missing_values > 0])

# Handle missing values
# For simplicity, we will fill missing numerical values with 0 and categorical values with 'Unknown'
numerical_cols = train.select_dtypes(include=[np.number]).columns
categorical_cols = train.select_dtypes(include=[object]).columns

train[numerical_cols] = train[numerical_cols].fillna(0)
train[categorical_cols] = train[categorical_cols].fillna('Unknown')


Missing values before cleaning:
dcoilwtico       955152
transactions     249117
type_y          2551824
locale          2551824
locale_name     2551824
description     2551824
transferred     2551824
dtype: int64
